In [2]:
!pip install pycaret-ts-alpha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 468 kB 4.7 MB/s 
     |████████████████████████████████| 69 kB 5.1 MB/s 
     |████████████████████████████████| 303 kB 54.9 MB/s 
     |████████████████████████████████| 6.5 MB 38.3 MB/s 
     |████████████████████████████████| 2.0 MB 35.8 MB/s 
     |████████████████████████████████| 11.2 MB 41.3 MB/s 
     |████████████████████████████████| 79.9 MB 114 kB/s 
     |████████████████████████████████| 3.3 MB 38.2 MB/s 
     |████████████████████████████████| 1.8 MB 53.5 MB/s 
     |████████████████████████████████| 134 kB 63.0 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 1.6 MB 49.4 MB/s 
     |████████████████████████████████| 957 kB 42.7 MB/s 
     |████████████████████████████████| 34.5 MB 14 kB/s 
     |████████████████████████████████| 9.8 MB 46.1 MB/s 
  Created wheel for pyod: filename=p

In [3]:
import pycaret

In [4]:
from pycaret.time_series import TSForecastingExperiment

In [5]:
!pip install kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from google.colab import files
uploads = files.upload()

Saving airquality.csv to airquality.csv


In [7]:
!pip install pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')


In [9]:
global_fig_settings = {"renderer": "notebook", "width": 1000, "height": 600}

In [10]:
data = pd.read_csv("airquality.csv")
data["index"] = pd.to_datetime(data["Date"] + " " + data["Time"])
data.drop(columns=["Date", "Time"], inplace=True)
data.replace(-200, np.nan, inplace=True)
data.set_index("index", inplace=True)

target = "CO(GT)"
exog_vars = ['NOx(GT)', 'PT08.S3(NOx)', 'RH']
include = [target] + exog_vars
data = data[include]
data.head()

,CO(GT),NOx(GT),PT08.S3(NOx),RH
index,,,,
2004-03-10 18:00:00,2.6,166.0,1056.0,48.9
2004-03-10 19:00:00,2.0,103.0,1174.0,47.7
2004-03-10 20:00:00,2.2,131.0,1140.0,54.0
2004-03-10 21:00:00,2.2,172.0,1092.0,60.0
2004-03-10 22:00:00,1.6,131.0,1205.0,59.6


In [11]:
FH=48
metric = "mase"
exclude = ["auto_arima", "bats", "tbats", "lar_cds_dt", "par_cds_dt"]

In [12]:
exp_auto = TSForecastingExperiment()

# enforce_exogenous=False --> Use multivariate forecasting when model supports it, else use univariate forecasting
exp_auto.setup(
    data=data, target=target, fh=FH, enforce_exogenous=False,
    numeric_imputation_target="ffill", numeric_imputation_exogenous="ffill",
    fig_kwargs=global_fig_settings, session_id=42
)

INFO:logs:PyCaret TSForecastingExperiment
INFO:logs:Logging name: ts-default-name
INFO:logs:ML Usecase: MLUsecase.TIME_SERIES
INFO:logs:version 3.0.0
INFO:logs:Initializing setup()
INFO:logs:self.USI: ab77
INFO:logs:self.variable_keys: {'data', '_ml_usecase', 'X_test', 'X_train_transformed', 'fh', 'log_plots_param', 'y_train', 'y_transformed', 'X_train', 'fold_generator', 'y_train_transformed', 'X_test_transformed', 'primary_sp_to_use', 'display_container', 'y_test', 'X_transformed', 'enforce_pi', 'approach_type', 'variable_keys', 'seed', 'seasonal_period', 'pipeline', '_all_metrics', 'logging_param', '_gpu_n_jobs_param', 'fold_param', 'html_param', 'y_test_transformed', 'strictly_positive', 'exogenous_present', 'idx', '_available_plots', 'n_jobs_param', 'fold_groups_param_full', 'enforce_exogenous', 'exp_id', 'y', 'master_model_container', '_all_models', 'seasonality_present', '_all_models_internal', 'X', 'USI', 'all_sp_values', 'index_type', 'experiment__', 'memory', 'gpu_param', 'ex

,Description,Value
0,session_id,42
1,Target,CO(GT)
2,Approach,Univariate
3,Exogenous Variables,Present
4,Data shape,"(9357, 4)"
5,Train data shape,"(9309, 4)"
6,Test data shape,"(48, 4)"
7,Fold Generator,ExpandingWindowSplitter
8,Fold Number,3
9,Enforce Prediction Interval,False


INFO:logs:setup() successfully completed in 10.37s...............


In [ ]:
best = exp_auto.compare_models(sort=metric, turbo=False, exclude=exclude)

In [ ]:
exp_auto.plot_model(best)

In [ ]:
final_auto_model = exp_auto.finalize_model(best)

In [ ]:
def safe_predict(exp, model):
    """Prediction wrapper for demo purposes."""
    try: 
        future_preds = exp.predict_model(model)
    except ValueError as exception:
        print(exception)
        exo_vars = exp.exogenous_variables
        print(f"{len(exo_vars)} exogenous variables (X) needed in order to make future predictions:\n{exo_vars}")
        
        
        exog_exps = []
        exog_models = []
        for exog_var in exog_vars:
            exog_exp = TSForecastingExperiment()
            exog_exp.setup(
                data=data[exog_var], fh=FH,
                numeric_imputation_target="ffill", numeric_imputation_exogenous="ffill",
                fig_kwargs=global_fig_settings, session_id=42
            )

            # Users can customize how to model future exogenous variables i.e. add
            # more steps and models to potentially get better models at the expense
            # of higher modeling time.
            best = exog_exp.compare_models(
                sort=metric, include=["arima", "ets", "exp_smooth", "theta", "lightgbm_cds_dt",]        
            )
            final_exog_model = exog_exp.finalize_model(best)

            exog_exps.append(exog_exp)
            exog_models.append(final_exog_model)

        # Step 2: Get future predictions for exog variables ----
        future_exog = [
            exog_exp.predict_model(exog_model)
            for exog_exp, exog_model in zip(exog_exps, exog_models)
        ]
        future_exog = pd.concat(future_exog, axis=1)
        future_exog.columns = exog_vars
        
        future_preds = exp.predict_model(model, X=future_exog)
    
    return future_preds

In [ ]:
future_preds = safe_predict(exp_auto, final_auto_model)
future_preds.plot()